In [ ]:
import lightgbm as lgb
import wandb
import pandas as pd


import wandb
run = wandb.init()
artifact = run.use_artifact('andrea-faviait/swiss-delay-prediction/model_k128i4ar:v0', type='model')
artifact_dir = artifact.download()
model = lgb.Booster(model_file=artifact_dir+'/model_ckpt_1690.txt') # https://wandb.ai/andrea-faviait/swiss-delay-prediction/artifacts/model/model_k128i4ar/f333850bd665f821e8ff/overview

In [ ]:
lgb.plot_importance(model)

In [ ]:
test_df = pd.read_csv('/Users/favea/Downloads/swiss-data/test_df.csv' ,index_col=0)
before_flights = pd.read_csv('/Users/favea/Downloads/swiss-data/previous_delayed_same_day.csv').drop('Unnamed: 0',axis=1)

In [ ]:
#test_df = test_df.merge(before_flights, left_on='id_flight', right_on='wh_fleg_leg_i')
test_df['departure_hour'] = test_df.scheduled_time_departure.apply(lambda x: pd.to_datetime(x).hour)
test_df['departure_minutes'] = test_df.scheduled_time_departure.apply(lambda x: pd.to_datetime(x).minute)

In [ ]:
columns_to_remove = ['ac_subtype','wh_fleg_leg_i','id_flight','carrier','season_trip','delay_minutes','scheduled_time_departure','year']

In [ ]:

test_df = test_df.drop(columns_to_remove,axis=1)
test_df = test_df.drop('ac_registration_code',axis=1)
x_test, y_test = test_df.drop('delayed',axis=1), test_df.loc[:,'delayed']
#x_train.carrier = x_train.carrier.astype('category')
x_test.origin = x_test.origin.astype('category')
x_test.destination = x_test.destination.astype('category')
#x_train.previous_is_delayed = x_train.previous_is_delayed.astype('category')
#x_train.ac_subtype = x_train.ac_subtype.astype('category')
#x_train.ac_registration_code = x_train.ac_registration_code.astype('category')
x_test.previous_is_delayed_same_day = x_test.previous_is_delayed_same_day.astype('category')

In [ ]:
y_test_pred = model.predict(x_test)

In [ ]:
x_test['predicted'] = (y_test_pred>=0.5).astype(int)
x_test['true'] = y_test

In [ ]:
x_test.sample(100)

In [ ]:
x_test.groupby(by=['true','predicted']).count().iloc[:,:1]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(x_test['true'],x_test['predicted']))